# Test socketdata Binance BTCUSDT
Méthodo pour récuperer les clés API et utiliser l'API Binance

## API KEYS

In [ ]:
import requests
import pandas as pd
import time
import os
from datetime import datetime
import subprocess
import json
import getpass


from binance.client import Client

In [2]:
def ensure_bw_session():
    """
    Vérifie si BW_SESSION est déjà défini.
    Sinon demande le mot de passe maître et déverrouille Bitwarden.
    """
    if "BW_SESSION" in os.environ and os.environ["BW_SESSION"]:
        return os.environ["BW_SESSION"]

    # Demande le mot de passe maître en mode caché
    master_password = getpass.getpass("Mot de passe Bitwarden: ")

    # Déverrouille Bitwarden et récupère le token
    result = subprocess.run(
        ["bw", "unlock", "--raw"],
        input=master_password + "\n",
        capture_output=True,
        text=True
    )

    if result.returncode != 0:
        raise RuntimeError(f"Erreur bw unlock: {result.stderr}")

    session = result.stdout.strip()
    os.environ["BW_SESSION"] = session
    return session

# Exemple d'utilisation
bw_session = ensure_bw_session()
print("✅ BW_SESSION actif :", bw_session[:10], "...")


✅ BW_SESSION actif : asAsg9mYbo ...


In [3]:
def list_bw_items(search=""):
    proc = subprocess.run(
        ["bw", "list", "items", "--search", search],
        capture_output=True, text=True, check=True, env=os.environ
    )
    return json.loads(proc.stdout)

items = list_bw_items("api")
for it in items:
    print("ID:", it["id"], "| Nom:", it.get("name"))

ID: def6d6fb-ad17-4d32-ad90-b35800d25ae7 | Nom: Binance API
ID: 8b02ca0a-6481-4ccb-a2da-b35b0143c670 | Nom: hermes_api2


In [ ]:
item_id = "c05e8e24-5a9c-4677-af2f-b35800d6d452"

proc = subprocess.run(
    ["bw", "get", "item", item_id],
    capture_output=True, text=True, check=True, env=os.environ
)

item = json.loads(proc.stdout)
print(json.dumps(item, indent=2)[:150])


In [ ]:
item_id = "c05e8e24-5a9c-4677-af2f-b35800d6d452"
name_id = "Binance API"
proc = subprocess.run(
    ["bw", "get", "item", name_id],
    capture_output=True, text=True, check=True, env=os.environ
)

item = json.loads(proc.stdout)
print(json.dumps(item, indent=2)[:50])  # juste pour vérifier, on n'affiche pas tout

In [5]:
name_id = "Binance API"
proc = subprocess.run(
    ["bw", "get", "item", name_id],
    capture_output=True, text=True, check=True, env=os.environ
)

credentiel = json.loads(proc.stdout)["login"]["password"]

In [8]:
key_name = "hermes_api2"
proc = subprocess.run(
    ["bw", "get", "item", key_name],
    capture_output=True, text=True, check=True, env=os.environ
)
publicKey = json.loads(proc.stdout)["login"]["password"]

In [ ]:
#!bw sync

##  python-binance
This is an unofficial Python wrapper for the Binance exchange REST API v3.
doc : https://python-binance.readthedocs.io/en/latest/index.html

In [ ]:
client = Client(publicKey, credentiel)

In [14]:
# get all symbol prices
prices = client.get_all_tickers()

In [15]:
prices

[{'symbol': 'ETHBTC', 'price': '0.03912000'},
 {'symbol': 'LTCBTC', 'price': '0.00098600'},
 {'symbol': 'BNBBTC', 'price': '0.00840000'},
 {'symbol': 'BTCUSDT', 'price': '117591.97000000'},
 {'symbol': 'ETHUSDT', 'price': '4599.77000000'},
 {'symbol': 'TRXBTC', 'price': '0.00000295'},
 {'symbol': 'XRPBTC', 'price': '0.00002642'},
 {'symbol': 'BNBUSDT', 'price': '987.81000000'},
 {'symbol': 'LTCUSDT', 'price': '118.25000000'},
 {'symbol': 'LTCBNB', 'price': '0.11620000'},
 {'symbol': 'XRPUSDT', 'price': '3.10580000'},
 {'symbol': 'XRPBNB', 'price': '0.00314500'},
 {'symbol': 'TRXBNB', 'price': '0.00034920'},
 {'symbol': 'TRXUSDT', 'price': '0.35130000'},
 {'symbol': 'NEOBTC', 'price': '0.00005771'},
 {'symbol': 'QTUMETH', 'price': '0.00054300'},
 {'symbol': 'GASBTC', 'price': '0.00002860'},
 {'symbol': 'BNBETH', 'price': '0.21450000'},
 {'symbol': 'LRCBTC', 'price': '0.00000083'},
 {'symbol': 'LRCETH', 'price': '0.00002118'},
 {'symbol': 'QTUMBTC', 'price': '0.00002124'},
 {'symbol': 'Z

In [16]:
# fetch weekly klines since it listed
klines = client.get_historical_klines("NEOBTC", Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2017")
klines

[[1756684800000,
  '0.00005840',
  '0.00005920',
  '0.00005840',
  '0.00005920',
  '46.27000000',
  1757289599999,
  '0.00273487',
  6,
  '17.15000000',
  '0.00101510',
  '0'],
 [1757289600000,
  '0.00005979',
  '0.00005979',
  '0.00005799',
  '0.00005831',
  '1459.92000000',
  1757894399999,
  '0.08598272',
  49,
  '1069.59000000',
  '0.06294405',
  '0'],
 [1757894400000,
  '0.00005748',
  '0.00005771',
  '0.00005647',
  '0.00005771',
  '202.94000000',
  1758499199999,
  '0.01156297',
  10,
  '32.56000000',
  '0.00186441',
  '0']]

In [23]:
status

{'status': 0, 'msg': 'normal'}

In [24]:
info = client.get_exchange_info()
info

{'timezone': 'UTC',
 'serverTime': 1758224289934,
 'rateLimits': [{'rateLimitType': 'REQUEST_WEIGHT',
   'interval': 'MINUTE',
   'intervalNum': 1,
   'limit': 6000},
  {'rateLimitType': 'ORDERS',
   'interval': 'SECOND',
   'intervalNum': 10,
   'limit': 50},
  {'rateLimitType': 'ORDERS',
   'interval': 'DAY',
   'intervalNum': 1,
   'limit': 160000},
  {'rateLimitType': 'RAW_REQUESTS',
   'interval': 'MINUTE',
   'intervalNum': 5,
   'limit': 61000}],
 'exchangeFilters': [],
 'symbols': [{'symbol': 'ETHBTC',
   'status': 'TRADING',
   'baseAsset': 'ETH',
   'baseAssetPrecision': 8,
   'quoteAsset': 'BTC',
   'quotePrecision': 8,
   'quoteAssetPrecision': 8,
   'baseCommissionPrecision': 8,
   'quoteCommissionPrecision': 8,
   'orderTypes': ['LIMIT',
    'LIMIT_MAKER',
    'MARKET',
    'STOP_LOSS',
    'STOP_LOSS_LIMIT',
    'TAKE_PROFIT',
    'TAKE_PROFIT_LIMIT'],
   'icebergAllowed': True,
   'ocoAllowed': True,
   'otoAllowed': True,
   'quoteOrderQtyMarketAllowed': True,
   'allo

In [25]:
info = client.get_all_tickers()
info

[{'symbol': 'ETHBTC', 'price': '0.03914000'},
 {'symbol': 'LTCBTC', 'price': '0.00098600'},
 {'symbol': 'BNBBTC', 'price': '0.00841100'},
 {'symbol': 'BTCUSDT', 'price': '117293.60000000'},
 {'symbol': 'ETHUSDT', 'price': '4590.80000000'},
 {'symbol': 'TRXBTC', 'price': '0.00000295'},
 {'symbol': 'XRPBTC', 'price': '0.00002637'},
 {'symbol': 'BNBUSDT', 'price': '986.73000000'},
 {'symbol': 'LTCUSDT', 'price': '118.07000000'},
 {'symbol': 'LTCBNB', 'price': '0.11620000'},
 {'symbol': 'XRPUSDT', 'price': '3.09450000'},
 {'symbol': 'XRPBNB', 'price': '0.00314500'},
 {'symbol': 'TRXBNB', 'price': '0.00034920'},
 {'symbol': 'TRXUSDT', 'price': '0.35040000'},
 {'symbol': 'NEOBTC', 'price': '0.00005771'},
 {'symbol': 'QTUMETH', 'price': '0.00054300'},
 {'symbol': 'GASBTC', 'price': '0.00002860'},
 {'symbol': 'BNBETH', 'price': '0.21500000'},
 {'symbol': 'LRCBTC', 'price': '0.00000083'},
 {'symbol': 'LRCETH', 'price': '0.00002118'},
 {'symbol': 'QTUMBTC', 'price': '0.00002124'},
 {'symbol': 'Z

In [26]:
trades = client.get_historical_trades(symbol='BTCUSDT')
trades

[{'id': 550855,
  'price': '117405.66000000',
  'qty': '0.00006000',
  'quoteQty': '7.04433960',
  'time': 1758223975536,
  'isBuyerMaker': True,
  'isBestMatch': True},
 {'id': 550856,
  'price': '117405.66000000',
  'qty': '0.00012000',
  'quoteQty': '14.08867920',
  'time': 1758223976012,
  'isBuyerMaker': True,
  'isBestMatch': True},
 {'id': 550857,
  'price': '117405.66000000',
  'qty': '0.00007000',
  'quoteQty': '8.21839620',
  'time': 1758223976227,
  'isBuyerMaker': True,
  'isBestMatch': True},
 {'id': 550858,
  'price': '117405.67000000',
  'qty': '0.00012000',
  'quoteQty': '14.08868040',
  'time': 1758223978937,
  'isBuyerMaker': False,
  'isBestMatch': True},
 {'id': 550859,
  'price': '117405.66000000',
  'qty': '0.00012000',
  'quoteQty': '14.08867920',
  'time': 1758223980246,
  'isBuyerMaker': True,
  'isBestMatch': True},
 {'id': 550860,
  'price': '117405.67000000',
  'qty': '0.00012000',
  'quoteQty': '14.08868040',
  'time': 1758223983026,
  'isBuyerMaker': False